In [ ]:
import requests
import pandas as pd
import sqlite3

In [3]:
# Define all functions

def fetch_cited_by(url, params=None):
    if params is None:
        params = {}
    params['per_page'] = 100  # items per page
    data_frames = []
    page = 1
    work_id = url.split('/')[-1]  # extract work id from URL

    with requests.Session() as session:
        while True:
            params['page'] = page
            response = session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            page_results = data.get('results', [])
            if page_results:
                # Temporarily drop abstracts before flattening
                abstracts = [r.pop('abstract_inverted_index', None) for r in page_results]
                df_page = pd.json_normalize(page_results, sep='_')
                df_page['abstract_inverted_index'] = abstracts
                # Tag each record with the seed_id
                df_page['seed_id'] = work_id
                data_frames.append(df_page)
            meta = data.get('meta', {})
            total_pages = meta.get('page_count', 1)
            print(f"Fetched page {page} of {total_pages} for cited_by work {work_id}")
            if page >= total_pages:
                break
            page += 1

    return pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()

def fetch_ref(chunks, params=None):
    if params is None:
        params = {}
    params['per_page'] = 50  # items per page for referenced works
    data_frames = []
    with requests.Session() as session:
        for seed_id, chunk in chunks:
            # Build the URL using the filter field "ids.openalex"
            url = f"https://api.openalex.org/works?filter=ids.openalex:{chunk}"
            page = 1
            while True:
                params['page'] = page
                response = session.get(url, params=params)
                response.raise_for_status()
                data = response.json()
                page_results = data.get('results', [])
                if page_results:
                    abstracts = [r.pop('abstract_inverted_index', None) for r in page_results]
                    df_page = pd.json_normalize(page_results, sep='_')
                    df_page['abstract_inverted_index'] = abstracts
                    df_page['seed_id'] = seed_id
                    data_frames.append(df_page)
                meta = data.get('meta', {})
                total_pages = meta.get('page_count', 1)
                print(f"Fetched page {page} of {total_pages} for chunk: {chunk}")
                if page >= total_pages:
                    break
                page += 1
    return pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()

def get_seed_data(seed_id):
    seed_url = f"https://api.openalex.org/works/{seed_id}"
    response = requests.get(seed_url)
    response.raise_for_status()
    return response.json()

def process_seed(seed_id):
    # Retrieve seed paper data
    seed_data = get_seed_data(seed_id)
    
    # Process referenced works into 50-item chunks
    referenced = seed_data.get('referenced_works', [])
    chunks = [
        (seed_id, "|".join(referenced[i:i+50]))
        for i in range(0, len(referenced), 50)
    ]
    
    print(f"Processing referenced works for seed {seed_id}")
    df_ref = fetch_ref(chunks)
    
    # Process cited_by works using the cited_by API URL
    cited_by_url = seed_data.get('cited_by_api_url')
    if cited_by_url:
        print(f"Processing cited_by works for seed {seed_id}")
        df_cited = fetch_cited_by(cited_by_url)
    else:
        print(f"No cited_by_api_url found for seed {seed_id}")
        df_cited = pd.DataFrame()
    
    return df_ref, df_cited

def process_seeds(seed_ids):
    master_ref_df = pd.DataFrame()
    master_cited_df = pd.DataFrame()
    
    for seed_id in seed_ids:
        print(f"\n=== Processing seed paper {seed_id} ===")
        df_ref, df_cited = process_seed(seed_id)
        master_ref_df = pd.concat([master_ref_df, df_ref], ignore_index=True)
        master_cited_df = pd.concat([master_cited_df, df_cited], ignore_index=True)
    
    # Tag the type of relationship for later chaining
    master_ref_df['relationship'] = 'referenced'
    master_cited_df['relationship'] = 'cited_by'
    
    # Combine both DataFrames
    combined_df = pd.concat([master_ref_df, master_cited_df], ignore_index=True)
    return combined_df

In [ ]:
# First level: Process the original seed papers
seed_ids = ['W85815303', 'W2001526706']  # Replace or add your seed paper IDs here
#'W85815303' Immerwahr & Foleno (2000)
#'W2001526706' #Doyle (2016)
first_level_df = process_seeds(seed_ids)
print("First-level results:")
print(first_level_df.head())

In [ ]:

# Extract unique work IDs from the first-level results (assuming the field 'id' holds the OpenAlex work ID)
# You can also filter by relationship if needed.
new_seed_ids = first_level_df['id'].dropna().unique().tolist()

# Optional: Exclude original seed IDs from the second-level seeds
second_level_seed_ids = [sid for sid in new_seed_ids if sid not in seed_ids]

print(f"Second-level seed IDs (new seeds): {second_level_seed_ids}")

In [ ]:
# Second level: Use these new seed IDs for further citation chaining
second_level_df = process_seeds(second_level_seed_ids)
print("Second-level results:")
print(second_level_df.head())

In [13]:
# Merge and clean
merged = pd.concat([first_level_df, second_level_df], ignore_index=True)

merged_clean = second_level_df.query("is_retracted != True") # Throw away retracted works
merged_clean = merged_clean[merged_clean['cited_by_count'] > 0]

merged_clean['id'] = merged_clean['id'].apply(lambda x: x.rsplit('/', 1)[-1] if isinstance(x, str) else x)
merged_clean['seed_id'] = merged_clean['seed_id'].apply(lambda x: x.rsplit('/', 1)[-1] if isinstance(x, str) else x)

In [19]:
# Create main dataframe
papers_cols = [
    'id', 'title', 'primary_location_landing_page_url', 'publication_date', 'publication_year',
    'language', 'type', 'open_access_is_oa', 'cited_by_count', 'fwci',
    'citation_normalized_percentile_is_in_top_1_percent',
    'citation_normalized_percentile_is_in_top_10_percent', 'relationship', 'seed_id'
]

papers = merged_clean[papers_cols]
papers = papers.drop_duplicates()

papers['citation_normalized_percentile_is_in_top_1_percent'] = papers['citation_normalized_percentile_is_in_top_1_percent'].astype(bool)
papers['citation_normalized_percentile_is_in_top_10_percent'] = papers['citation_normalized_percentile_is_in_top_10_percent'].astype(bool)


In [15]:
# Create authors dataframe
authors = pd.json_normalize(
    merged_clean.to_dict(orient='records'),
    record_path='authorships',
    meta=['id']
)

authors_cols = [
    'id', 'author.id', 'author.display_name', 'author.orcid',
    'raw_author_name', 'author_position', 'is_corresponding'
]

authors_clean = authors[authors_cols].drop_duplicates(subset=['author.id'])
authors_clean['author.id'] = authors_clean['author.id'].apply(lambda x: x.rsplit('/', 1)[-1] if isinstance(x, str) else x)
authors_clean['author.orcid'] = authors_clean['author.orcid'].apply(lambda x: x.rsplit('/', 1)[-1] if isinstance(x, str) else x)

In [ ]:
abstract_records = []
# Loop over each paper in the merged_clean dataframe
for idx, row in merged_clean.iterrows():
    paper_id = row['id']
    abstract_dict = row.get('abstract_inverted_index')
    # Make sure we have a dictionary
    if isinstance(abstract_dict, dict):
        # For each word in the abstract dictionary, create a new record
        for word, positions in abstract_dict.items():
            abstract_records.append({
                'paper_id': paper_id,
                'word': word,
                'positions': positions  # this can be a list, or you can convert it to a string if needed
            })

# Create a new DataFrame from these records
abstracts = pd.DataFrame(abstract_records)
print(abstracts.head())

In [17]:
bridge = authors_clean[['id', 'author.id']].drop_duplicates()

In [20]:
import sqlite3

# Connect to (or create) the SQLite database
conn = sqlite3.connect("openalex.db")
cursor = conn.cursor()

# -------------------------------
# Create the 'papers' table
# -------------------------------
cursor.execute("DROP TABLE IF EXISTS papers;")
cursor.execute("""
CREATE TABLE papers (
    id TEXT PRIMARY KEY,
    title TEXT,
    primary_location_landing_page_url TEXT,
    publication_date TEXT,
    publication_year INTEGER,
    language TEXT,
    type TEXT,
    open_access_is_oa BOOLEAN,
    cited_by_count INTEGER,
    fwci REAL,
    citation_normalized_percentile_is_in_top_1_percent BOOLEAN,
    citation_normalized_percentile_is_in_top_10_percent BOOLEAN
);
""")

# -------------------------------
# Create the 'authors' table
# -------------------------------
# Here, 'id' represents the paper_id from the merged_clean DataFrame.
# We'll rename "author.id" to "author_id" and "author.display_name" to "author_display_name"
cursor.execute("DROP TABLE IF EXISTS authors;")
cursor.execute("""
CREATE TABLE authors (
    paper_id TEXT,
    author_id TEXT,
    author_display_name TEXT,
    author_orcid TEXT,
    raw_author_name TEXT,
    author_position TEXT,
    is_corresponding BOOLEAN,
    PRIMARY KEY (author_id),
    FOREIGN KEY (paper_id) REFERENCES papers(id)
);
""")


# -------------------------------
# Create the 'abstracts' table
# -------------------------------
cursor.execute("DROP TABLE IF EXISTS abstracts;")
cursor.execute("""
CREATE TABLE abstracts (
    paper_id TEXT,
    word TEXT,
    positions TEXT,
    record_id INTEGER PRIMARY KEY AUTOINCREMENT,
    FOREIGN KEY (paper_id) REFERENCES papers(id)
);
""")

# -------------------------------
# Create the 'bridge' table
# -------------------------------
# This table serves as a many-to-many join between papers and authors.
cursor.execute("DROP TABLE IF EXISTS bridge;")
cursor.execute("""
CREATE TABLE bridge (
    paper_id TEXT,
    author_id TEXT,
    PRIMARY KEY (author_id),
    FOREIGN KEY (paper_id) REFERENCES papers(id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);
""")

conn.commit()

# Now, insert data from your DataFrames.
# Make sure to adjust column names in your DataFrames if necessary.
# For example, if your authors_clean DataFrame still has columns "author.id", rename them:
authors_clean = authors_clean.rename(columns={
    'id': 'paper_id',
    'author.id': 'author_id',
    'author.display_name': 'author_display_name',
    'author.orcid': 'author_orcid'
})

bridge = bridge.rename(columns={
    'id': 'paper_id',
    'author.id': 'author_id',
})

abstracts['positions'] = abstracts['positions'].apply(
    lambda x: ",".join(map(str, x)) if isinstance(x, list) else str(x)
)

# Insert the papers data (from merged_clean)
papers.to_sql("papers", conn, if_exists="append", index=False)

# Insert the authors data (authors_clean)
authors_clean.to_sql("authors", conn, if_exists="append", index=False)

# Insert the abstracts data (abstracts_df)
abstracts.to_sql("abstracts", conn, if_exists="append", index=False)

# Insert the bridge table (bridge_table)
bridge.to_sql("bridge", conn, if_exists="append", index=False)

conn.commit()
conn.close()

In [ ]:
papers.info()
authors_clean.info()
bridge.info()
abstracts.info()